In [1]:
import jupyter_black
from IPython.display import display


jupyter_black.load(line_length=999)

In [ ]:
import os
import torch
import torchvision

In [ ]:
class Person:
    def __init__(self, sid: str):
        self.sid = sid
        self.path = os.path.join(os.getcwd(), "dataset", sid)
        self.data = [os.path.join(self.path, item) for item in os.listdir(self.path)]
        self._index = 0

    def __iter__(self):
        self._index = 0
        return self

    def __next__(self):
        if self._index < len(self.data):
            item = self.data[self._index]
            self._index += 1
            return item
        else:
            raise StopIteration